### 第八章 数据规整：连接、联合和重塑

#### 8.1 层次化索引

层次化索引是pandas的一项重要功能，它使你能在一个轴上拥有多个索引级别。抽象点说，它使你能以低维度形式处理高维度数据。

In [2]:
import numpy as np
import pandas as pd

data = pd.Series(np.random.uniform(size = 9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.436924
   2    0.892663
   3    0.598365
b  1    0.944753
   3    0.981397
c  1    0.292634
   2    0.328474
d  2    0.512804
   3    0.213377
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data["b"]

1    0.944753
3    0.981397
dtype: float64

In [5]:
data["b":"c"]

b  1    0.944753
   3    0.981397
c  1    0.292634
   2    0.328474
dtype: float64

In [6]:
data[["b", "d"]]

b  1    0.944753
   3    0.981397
d  2    0.512804
   3    0.213377
dtype: float64

In [7]:
data.loc[:, 2]

a    0.892663
c    0.328474
d    0.512804
dtype: float64

In [8]:
# 为啥结果中有NaN？
data.unstack()

,1,2,3
a,0.436924,0.892663,0.598365
b,0.944753,NaN,0.981397
c,0.292634,0.328474,NaN
d,NaN,0.512804,0.213377


In [9]:
data.unstack().stack()

a  1    0.436924
   2    0.892663
   3    0.598365
b  1    0.944753
   3    0.981397
c  1    0.292634
   2    0.328474
d  2    0.512804
   3    0.213377
dtype: float64

In [10]:
# 对于DataFrame, 每个轴都可以有分层索引
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], columns=[['A', 'A', 'B'], ['a', 'b', 'a']])
frame

A       B
     a   b   a
a 1  0   1   2
  2  3   4   5
b 1  6   7   8
  2  9  10  11

In [11]:
frame.index.nlevels

2

In [12]:
frame["A"]

a   b
a 1  0   1
  2  3   4
b 1  6   7
  2  9  10

In [13]:
frame.index.names = ["key1", "key2"]
frame.swaplevel("key1", "key2")

A       B
           a   b   a
key2 key1           
1    a     0   1   2
2    a     3   4   5
1    b     6   7   8
2    b     9  10  11

In [14]:
frame.sort_index(level=1)

A       B
           a   b   a
key1 key2           
a    1     0   1   2
b    1     6   7   8
a    2     3   4   5
b    2     9  10  11

In [15]:
frame.groupby(level="key1").sum()

A       B
       a   b   a
key1            
a      3   5   7
b     15  17  19

In [16]:
frame.columns.names = ["state", "color"]
frame.T.groupby(level="color").sum()

key1   a      b    
key2   1  2   1   2
color              
a      2  8  14  20
b      1  4   7  10

In [17]:
frame = pd.DataFrame({
    "a": range(7),
    "b": range(7, 0, -1),
    "c": ["one", "one", "one", "two", "two", "two", "two"],
    "d": [0, 1, 2, 0, 1, 2, 3]
})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [18]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [19]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


#### 8.2 联合与合并数据集

In [21]:
df1 = pd.DataFrame({
    "key": ["b", "b", "a", "c", "a", "a", "b"],
    "data1": range(7)
})
df2 = pd.DataFrame({
    "key": ["a", "b", "d"],
    "data2": range(3)
})
print(df1)
print(df2)
pd.merge(df1, df2, on="key")

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [22]:
df3 = pd.DataFrame({
    "lkey": ["b", "b", "a", "c", "a", "a", "b"],
    "data1": pd.Series(range(7), dtype="Int64")
})

df4 = pd.DataFrame({
    "rkey": ["a", "b", "d"],
    "data2": pd.Series(range(3), dtype="Int64")
})

pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


默认情况，pandas.merge 是“内连接”，结果中的键是交集。其他方式还有“左连接”、“右连接”、“外连接”。

In [23]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


In [24]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [26]:
left = pd.DataFrame({
    "key1": ["foo", "foo", "bar"],
    "key2": ["one", "two", "one"],
    "lval": [1, 2, 3]
})
right = pd.DataFrame({
    "key1": ["foo", "foo", "bar", "bar"],
    "key2": ["one", "one", "one", "two"],
    "rval": [4, 5, 6, 7]
})
pd.merge(left, right, on=["key1", "key2"], how="inner")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [ ]:
# 将多个键当做元组处理，并且将元组作为单个链接键使用
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3.0,6.0
1,bar,two,NaN,7.0
2,foo,one,1.0,4.0
3,foo,one,1.0,5.0
4,foo,two,2.0,NaN


In [28]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [29]:
left1 = pd.DataFrame({
    "key": ["a", "b", "a", "a", "b", "c"],
    "value": range(6)
})
right1 = pd.DataFrame({
    "group_val": [3.5, 7],
}, index=["a", "b"])

print(left1)
print(right1)

pd.merge(left1, right1, left_on="key", right_index=True)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [31]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [32]:
arr = np.arange(12).reshape((3, 4))
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [34]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [35]:
s1 = pd.Series([0, 1], index=["a", "b"])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [36]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [37]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [38]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [39]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [41]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [42]:
result.unstack(level=0)

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [43]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [44]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [45]:
df1 = pd.DataFrame(
    np.arange(6).reshape(3, 2),
    index=["a", "b", "c"],
    columns=["one", "two"]
)
df2 = pd.DataFrame(
    np.arange(4).reshape(2, 2),
    index=["a", "c"],
    columns=["three", "four"]
)

pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    0.0  1.0
b      2   3    NaN  NaN
c      4   5    2.0  3.0

In [46]:
pd.concat([df1, df2], axis="columns", keys=["key1", "key2"], names=["upper", "lower"])

upper key1      key2     
lower  one two three four
a        0   1   0.0  1.0
b        2   3   NaN  NaN
c        4   5   2.0  3.0

In [47]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)), columns=list('abcd'))
df2 = pd.DataFrame(np.random.standard_normal((2, 3)), columns=list('bda'))

pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.349876,0.491651,0.429984,0.101382
1,-0.367999,1.628873,1.577722,2.250780
2,-1.480745,-0.448509,-0.126925,-0.814484
3,1.541717,1.966151,NaN,-1.289363
4,1.759858,-0.119751,NaN,0.272747


In [48]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=["f", "e", "d", "c", "b", "a"])
b = pd.Series([0., np.nan, 2., np.nan, 4., np.nan], index=["a", "b", "c", "d", "e", "f"])

np.where(pd.isna(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [49]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    NaN
e    2.5
f    NaN
dtype: float64

In [50]:
df1 = pd.DataFrame({
    "a": [1., np.nan, 5., np.nan],
    "b": [1., 2., 5., np.nan],
    "c": range(2, 18, 4)
})
df2 = pd.DataFrame({
    "a": [5., 4., np.nan, 3., 7.],
    "b": [5., np.nan, 3., 4., 2.],
})

df1.combine_first(df2)

,a,b,c
0,1.0,1.0,2.0
1,4.0,2.0,6.0
2,5.0,5.0,10.0
3,3.0,4.0,14.0
4,7.0,2.0,NaN


#### 8.3 重塑和透视

In [52]:
# 重新排列表格型数据，这些操作称为重塑和透视
data = pd.DataFrame(
    np.arange(6).reshape((2, 3)),
    index=pd.Index(['Ohio', 'Colorado'], name='state'),
    columns=pd.Index(['one', 'two', 'three'], name='number'),
)
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [54]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [55]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [56]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [57]:
s1 = pd.Series([1, 2, 3, 4], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    1
     b    2
     c    3
     d    4
two  c    4
     d    5
     e    6
dtype: int64

In [58]:
data2.unstack()

,a,b,c,d,e
one,1.0,2.0,3.0,4.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [59]:
data2.unstack().stack(dropna=False)

/var/folders/7v/zmfj4myd5gj8ttpbq6ynmdm80000gn/T/ipykernel_3467/3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a    1.0
     b    2.0
     c    3.0
     d    4.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [60]:
data = pd.read_csv("examples/macrodata.csv")
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2
...,...,...,...,...,...
198,2008,3,13324.600,-3.16,6.0
199,2008,4,13141.920,-8.79,6.9
200,2009,1,12925.410,0.94,8.1
201,2009,2,12901.504,3.37,9.2


In [61]:
periods = pd.PeriodIndex(
    year=data.pop("year"),
    quarter=data.pop("quarter"),
    name="date"
)
periods

/var/folders/7v/zmfj4myd5gj8ttpbq6ynmdm80000gn/T/ipykernel_3467/3183764449.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [62]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [63]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [65]:
long_data = data.stack().reset_index().rename(columns={0:'value'})
long_data.head()

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340


In [66]:
pivoted = long_data.pivot(index="date", columns="item", values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [68]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data.head()

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,1.946596
1,1959-01-01,infl,0.000,0.997489
2,1959-01-01,unemp,5.800,0.440870
3,1959-04-01,realgdp,2778.801,1.140191
4,1959-04-01,infl,2.340,0.043385


In [70]:
pivoted = long_data.pivot(index='date', columns='item')
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.997489  1.946596  0.440870
1959-04-01  2.34  2778.801   5.1  0.043385  1.140191 -0.890945
1959-07-01  2.74  2775.488   5.3  0.835666 -0.545181 -0.799671
1959-10-01  0.27  2785.204   5.6  0.804018 -1.607644 -0.187093
1960-01-01  2.31  2847.699   5.2 -0.147646  0.777569  0.454178

In [72]:
df = pd.DataFrame({
    "key": ["foo", "bar", "baz"],
    "A": [1, 2, 3],
    "B": [4, 5, 6],
    "C": [7, 8, 9]
})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [74]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [77]:
reshaped = melted.pivot(index='key', columns='variable', values='value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [78]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [79]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [82]:
print(df)
pd.melt(df, value_vars=['A', 'B', 'C'])

   key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9


,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [81]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
